In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
import os
import sys

drive.mount('/content/drive', force_remount=True)

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = "cs231n/lora_w_diffusion-main/"
assert FOLDERNAME is not None, "[!] Enter the foldername."
PROJECT_PATH = f"/content/drive/My Drive/{FOLDERNAME}"
sys.path.append(PROJECT_PATH)

# Change working directory
os.chdir(PROJECT_PATH)

# Confirm
print("✅ Current working directory:", os.getcwd())
print("📁 Contents:", os.listdir('.'))

# 💡 LoRA Training on Stable Diffusion
This notebook trains custom LoRA adapters on Stable Diffusion using your own image-caption pairs.


In [ ]:
!pip install -q diffusers transformers accelerate torchvision safetensors kornia wandb torchmetrics[image]

## Data Preparation

In [ ]:
# Hyper params
lora_rank = 2
lora_alpha = 1
epochs = 1
lr = 1e-4
dropout = 0.1
dataset = "data_chinese"
in_training_prompt = "米芾 行书"
num_img_to_generate = 3
agumentation = True
conv_lora = True
device = "cuda"

run_name = f"dataset_{dataset}_rank_{lora_rank}_alpha_{lora_alpha}_epochs_{epochs}_conv_{int(conv_lora)}_lr_{lr:.0e}_dropout_{dropout}_augment_{int(agumentation)}"
print(run_name)

In [ ]:
import os

import torch
from torch.utils.data import Dataset, DataLoader
from dataset import ImageTextDataset

dataset = ImageTextDataset(dataset)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

# sanity check for datasets
print(f"Number of image-caption pairs: {len(dataset)}")

In [ ]:
# preview a couple of images.
from matplotlib import pyplot as plt
import warnings
import re

# This regex will match messages containing "Glyph" and "missing from font"
warnings.filterwarnings("ignore", category=UserWarning, message=r"Glyph .* missing from font\(s\) DejaVu Sans\.")

for i in range(min(2, len(dataset))):  # Show up to 3 examples
    image, caption = dataset[i]
    plt.imshow(image.permute(1, 2, 0).numpy() * 0.5 + 0.5)  # Undo normalization
    plt.axis("off")
    plt.title(caption)
    plt.show()


## Training Logic for Lora Finetuning

In [ ]:
import wandb
import os
os.environ["WANDB_API_KEY"] = "20e7c4be307028d246fbff111508b75d9eaab1ee"
wandb.login()

wandb.init(
    project="stable-diffusion-calligraphy",
    name=run_name,  # give each experiment a unique name
    config={
        "lora_rank": lora_rank,
        "lora_alpha": lora_alpha,
        "lr": lr,
        "epochs": epochs,
        "augmentations": False,
        "conv_lora": conv_lora,
    }
)

In [ ]:
from lora import LoRALinear, LoRAConv2d
from patch_unet import patch_unet_with_lora, conv_filter

from diffusers import StableDiffusionPipeline
import torch.nn as nn


pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float32).to(device)

pipe.vae.requires_grad_(False)
pipe.text_encoder.requires_grad_(False)
pipe.unet.requires_grad_(False)

# ADD LORA
patch_unet_with_lora(pipe.unet, r=lora_rank, alpha=lora_alpha, dropout=dropout, conv_filter=None)

pipe.unet.to(device)  # Move after patching

# unfreeze lora weights
for module in pipe.unet.modules():
    if isinstance(module, (LoRALinear, LoRAConv2d)):
        for p in module.parameters():
            p.requires_grad = True

optimizer = torch.optim.Adam([p for p in pipe.unet.parameters() if p.requires_grad], lr=lr)


In [ ]:
# Loads the CLIP model and processor
from clip import calculate_clip_score
from transformers import CLIPProcessor, CLIPModel

clip_model_name ="openai/clip-vit-large-patch14" # Consistent with CompVis/stable-diffusion-v1-4
clip_model = CLIPModel.from_pretrained(clip_model_name).to(device)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

In [ ]:
def generate_and_log_image(pipe, clip_model, clip_processor, device, prompt, epoch):
    """
    Generate an image from a prompt using the pipeline, calculate CLIP score,
    and log the result to Weights & Biases.

    Args:
        pipe: The diffusion pipeline.
        clip_model: The CLIP model for scoring.
        clip_processor: The processor for CLIP input.
        device: torch.device to use.
        prompt (str): Text prompt to generate image from.
        epoch (int): Current training epoch, for logging.
    """
    with torch.no_grad():
        image = pipe(prompt, num_inference_steps=30).images[0]

    # Calculate CLIP score
    clip_score = calculate_clip_score(image, prompt, clip_model, clip_processor, device)

    # Log image, score, and prompt to wandb
    wandb.log({
        "epoch": epoch,
        "generated_image": wandb.Image(image, caption=f"{prompt} | score: {clip_score:.3f}"),
        "clip_score": clip_score,
    })

    return image, clip_score  # optionally return for other use

In [ ]:
import torch
from augment import augment
from patch_unet import save_lora_weights, load_lora_weights
import torch.nn as nn

# Save model run with different hyper parameter separately
model_path = f"model_weights/{run_name}.pth"

# training
for epoch in range(epochs):
    for i, (images, captions) in enumerate(loader):
        # Keep images in float32
        images = images.to(device)

        images = augment(images)

        text_input = pipe.tokenizer(captions, padding="max_length", max_length=77, return_tensors="pt").to(device)
        text_embeds = pipe.text_encoder(**text_input).last_hidden_state.to(device)  # stays float32

        with torch.no_grad():
            vae_output = pipe.vae.encode(images)
        latents = vae_output.latent_dist.sample().to(device) * 0.18215  # float32

        noise = torch.randn_like(latents).to(device)  # float32 noise
        timesteps = torch.randint(0, 1000, (latents.shape[0],), device=device).long()

        noisy_latents = pipe.scheduler.add_noise(latents, noise, timesteps)

        noise_pred = pipe.unet(noisy_latents, timesteps, encoder_hidden_states=text_embeds).sample  # float32

        loss = nn.MSELoss()(noise_pred, noise)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print(f"Epoch {epoch} Step {i} Loss: {loss.item():.4f}")
        wandb.log({
            "epoch": epoch,
            "step": i,
            "loss": loss.item(),
            "lr": optimizer.param_groups[0]["lr"],
        })

    # Log one of the input images
    sample_image = (images[0].cpu().numpy() * 0.5 + 0.5).transpose(1, 2, 0)
    wandb.log({
        "input_image": wandb.Image(sample_image, caption=captions[0])
    })

    # Generate and log output image + clip score
    generate_and_log_image(pipe, clip_model, clip_processor, device, in_training_prompt, epoch)


save_lora_weights(pipe.unet, path=model_path)

## Generate images with Trained LoRA

In [ ]:
import os
import torch
import wandb

# Load LoRA weights and patch UNet
# model_path = "lora_weights.pth"
model_path = f"model_weights/{run_name}.pth"

state_dict = torch.load(model_path, map_location=device)
patch_unet_with_lora(pipe.unet, r=lora_rank, alpha=lora_alpha)
pipe.unet.load_state_dict(state_dict, strict=False)
pipe.unet.eval()

# Setup output
output_dir = "generated_images"
os.makedirs(output_dir, exist_ok=True)

# Define prompts
chinese_prompts = [
    ("米芾 行书", "米芾_行书_generated"),
    ("褚遂良 楷书", "褚遂良_楷书_generated"),
    ("楷书", "楷书_generated")
]

# Define prompts
english_prompts = [
    ("mi fu chinese caligraphy xingshu", "米芾_caligraphy_generated"),
    ("chu suiliang chinese caligraphy kaishu", "褚遂良_caligraphy_generated"),
    ("chinese caligraphy kaishu", "kaishu_caligraphy_generated")
]

baseline_prompts = [
    # not in training set
    ("邓石如 隶书", "邓石如_隶书_generated"),
    ("deng shiru chinese caligraphy lishu", "邓石如_lishu_caligraphy_generated"),

    # some english non caligraphy prompt
    ("a beautiful sunset in California", "sunset_generated"),
    ("a cute cat", "cute_cat_generated"),
]

all_propmpts = chinese_prompts + english_prompts + baseline_prompts

def generate_log_display_images(prompt, filename_prefix):
    for i in range(num_img_to_generate):
        image = pipe(prompt, num_inference_steps=30).images[0]
        clip_score = calculate_clip_score(
            image, prompt, clip_model, clip_processor, device
        )

        wandb.log({
            "prompt": prompt,
            "clip_score": clip_score,
            "output image": wandb.Image(image, caption=f"{prompt} | score: {clip_score:.3f}")
        })
        print (f"{prompt} | clip_score {clip_score}")
        image_path = os.path.join(output_dir, f"{filename_prefix}_{i}.png")
        image.save(image_path)
        image.show()
        display(image)  # for inline notebook display

# Run generation for all prompts
for prompt_text, filename_prefix in all_propmpts:
    generate_log_display_images(prompt_text, filename_prefix)


##[Optional] Visualize Layers

In [ ]:
import torch.nn as nn

print("Available Conv2d layers in the UNet:")
for name, module in pipe.unet.named_modules():
    if isinstance(module, nn.Conv2d):
        print(f"- {name} (Output shape usually depends on input size)")

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from patch_unet import remove_all_hooks, register_hooks, get_activation

activations = {}
# --- Choose Layers and Register Hooks ---
layers_to_visualize = [
    'down_blocks.0.resnets.0.conv1',
    'up_blocks.0.resnets.0.conv1',
    'mid_block.resnets.0.conv2'
]

# Clear previous hooks/activations if running multiple times
remove_all_hooks()
activations.clear()

register_hooks(pipe.unet, layers_to_visualize)

In [ ]:
prompt = "米芾 行书"
generator = torch.Generator(device=device).manual_seed(42) # Use a seed for reproducibility

print("Running inference...")
image = pipe(prompt, num_inference_steps=30, generator=generator).images[0]
print("Inference complete.")

# It's crucial to remove hooks *after* inference
remove_all_hooks()

In [ ]:
from patch_unet import plot_activations

for layer_name in layers_to_visualize:
    # Choose batch_index=0 or batch_index=1 to see the difference
    plot_activations(layer_name, num_cols=16, scale=1.5, batch_index=0)
    #plot_activations(layer_name, num_cols=16, scale=1.5, batch_index=1)

##Merge Lora weights back to the base stable diffusion model

In [ ]:
wandb.finish()